In [15]:
import queue
import re
from urllib.parse import urlparse
from urllib.parse import urljoin

from urllib.parse import urlsplit  # splitting out the url - netloc and path
import time  # add download delay - 5 seconds 


In [16]:

from bs4 import BeautifulSoup
import requests
from selenium import webdriver


In [17]:


def is_absolute(url):
    """Determine whether URL is absolute."""
    return bool(urlparse(url).netloc)



In [18]:

options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(chrome_options=options)



/Users/ianchriste/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [19]:
# List to hold email addresses and q for the Queue of urls

email_addresses = []

q = queue.Queue()

q.put("https://www.stevens.edu/school-business/faculty")



In [20]:
# create
final_list = []  # capture all links in this list
email_total =[]  # capture all emails in this list


In [21]:


link_list = []  # To Avoid visiting any page more than once



for i in range(8):
    i = 1 + i
    print("loop: {}".format(i))
    url = q.get()

    # r = requests.get(url)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract all email addresses.
    # print(soup.get_text())
    email_addresses += re.findall("\S+@stevens.edu", soup.get_text())
    email_addresses = list(set(email_addresses))

    links = soup.find_all('a')
    links = set(links)
    

    for link in links:
        if link not in link_list:
            link_list.append(link)
        
    for link in link_list:
        u = link.get('href')
        if not is_absolute(u):
            u = urljoin(url, u)
        if "stevens.edu" in u:
            q.put(u)
            
    time.sleep(10)   # set your crawl delay to 10 seconds
    
    final_list.append(u) # final list 
    
    email_total.append(email_addresses)
    
    print("Queue size: {}".format(q.qsize()))
    print("# email addresses: {}".format(len(email_addresses)))

loop: 1
Queue size: 525
# email addresses: 121
loop: 2
Queue size: 1095
# email addresses: 122
loop: 3
Queue size: 1666
# email addresses: 123
loop: 4
Queue size: 2273
# email addresses: 123
loop: 5
Queue size: 2880
# email addresses: 123
loop: 6
Queue size: 3274
# email addresses: 123
loop: 7
Queue size: 3928
# email addresses: 123
loop: 8
Queue size: 4622
# email addresses: 124


In [22]:
directory = []    # directory structure that reflectes the hierarchy
for link in final_list:
    parts = urlsplit(link)
    b = parts.scheme    # url split - scheme
    c = parts.path      # url split - path
    d = parts.netloc    # url split - netloc
    directory.append(b)
    directory.append(c)
    directory.append(d)

In [23]:
print(directory)

['http', '/facultyprofile/', 'web.stevens.edu', 'https', '/events/stevens-graduate-open-house-7', 'www.stevens.edu', 'http', '/facultyprofile/', 'web.stevens.edu', 'http', '/', 'www.panynj.gov', 'http', '/', 'www.panynj.gov', 'http', '/', 'www.panynj.gov', 'https', '/bM35YWydMa', 't.co', 'https', '/directory/development-and-alumni-engagement/stay-informed/contact-us', 'www.stevens.edu']


In [24]:
with open("email.txt", "w+") as f:
    for e in email_addresses:
        f.write(e + "\n")


In [25]:
with open("directory.txt", "w+") as f:
    for e in directory:
        f.write(e + "\n")
